In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.stats import skew, kurtosis, mode
from scipy.spatial import KDTree
from sklearn.cluster import DBSCAN
from ipywidgets import interact
import matplotlib.patches as mpatches
import plotly.graph_objects as go
from scipy.ndimage import convolve


In [2]:
def find_parts(directory_path, str = "AdaptiveZ_10mm"):
    parts_paths = []

    for root, dirs, files in os.walk(directory_path):
        for folder in dirs:
            folder = os.path.join(root,folder)
            if (str in folder) and folder.endswith("_4"):
                parts_paths.append(folder)
    parts_paths.sort(reverse=True)
    return parts_paths


base_path = r"\\192.168.1.100\CoreScan3-2\Acquisitions\RnD\XRF\CH\Macassa_clearance"

paths = find_parts(base_path)

In [ ]:
def get_point_cloud(file_path):
    coords_file = os.path.join(file_path, '.component_parameters.txt')
    with open(coords_file) as file:
            lines = file.readlines()
                
            for line in lines:
                if "XRAY_DPP[Acquisition]#0.Y.Start:" in line:
                    y_offset = (float)(line.split("XRAY_DPP[Acquisition]#0.Y.Start:")[1].strip())
                elif "XRAY_DPP[Acquisition]#0.X.Start:" in line:
                    x_start = (float)(line.split("XRAY_DPP[Acquisition]#0.X.Start:")[1].strip())
                elif "XRAY_DPP[Acquisition]#0.X.Stop:" in line:
                    x_stop = (float)(line.split("XRAY_DPP[Acquisition]#0.X.Stop:")[1].strip())


    point_cloud = []

    if os.path.isdir(file_path):
        lidar_files = [fn for fn in os.listdir(
            file_path) if fn.endswith('.bpc')]
        if any(lidar_files):
            lidar_filename = file_path + os.sep + lidar_files[0]

    data = np.fromfile(lidar_filename, dtype=np.float32)
    point_cloud = data.reshape(-1, 3)  # to xyz

    ff = ~np.isnan(point_cloud).any(axis=1)
    point_cloud = point_cloud[ff, ...]

    #point_cloud[:, 1] = point_cloud[:, 1] - float(y_offset)

    print(f"{file_path} is loaded. \n# of point {point_cloud.shape[0]}")


    matrix_file = (os.path.join(file_path, ".XRAY_DPP_001.lidar2xrf"))
    with open(matrix_file, 'r') as file:
        lines = file.readlines()

    transformation_matrix = np.array([list(map(float, line.strip().split(","))) for line in lines])

    num_points = point_cloud.shape[0]

    homogeneous_points = np.hstack((point_cloud, np.ones((num_points, 1))))
    transformed_points = homogeneous_points @ transformation_matrix.T
    point_cloud = transformed_points[:, :3]


    def trim_cloud(data):
        data = data[data[:, 2] > 0]
        data = data[
            (data[:,0] >= x_stop) & 
            (data[:,0] <= x_start) 
            ]
        return data
    
  
    def trim_y(data, y_span=16):
        data = data[(data[:,1] >= -y_span) & (data[:,1] <= y_span)]
        return data
    
    
    
    point_cloud = trim_cloud(point_cloud)
    point_cloud = trim_y(point_cloud)
    return point_cloud, x_start, x_stop


def join_y_values(data):
        y_vals = np.sort(np.unique(data[:,1]))
        print(len(y_vals))
        unique_y = []
        previous_value = y_vals[0]
        unique_y.append(previous_value)
        tolerance = 1e-1
    
        for value in y_vals[1:]:
            if abs(value - previous_value) > tolerance:  
                unique_y.append(value)  
                previous_value = value
        print(len(unique_y))
        
        data[:,1] = [unique_y[np.argmin(np.abs(unique_y - y))] for y in data[:,1]]
        return data


In [ ]:
point_clouds = []

x_offset = 0

for path in paths:
    temp_cloud, x_start, x_stop = get_point_cloud(path)
    x_offset -= x_stop
    temp_cloud[:,0] += x_offset
    x_offset += x_start
    point_clouds.append(temp_cloud)
   
point_cloud = np.vstack(point_clouds)
x_points = np.unique(point_cloud[:,0])
print(f"total # of point {point_cloud.shape[0]}")

point_tree = KDTree(point_cloud[:,:2])

In [ ]:
def join_y_values(data):
        y_vals = np.sort(np.unique(point_cloud[:,1]))
        print(len(y_vals))
        unique_y = []
        previous_value = y_vals[0]
        unique_y.append(previous_value)
        tolerance = 1e-1
    
        for value in y_vals[1:]:
            if abs(value - previous_value) > tolerance:  
                unique_y.append(value)  
                previous_value = value
        print(len(unique_y))
        
        data[:,1] = [unique_y[np.argmin(np.abs(unique_y - y))] for y in data[:,1]]
        return data


point_cloud = join_y_values(point_cloud)

In [ ]:
def get_vectors(data1, data2, x_vals, y_span=10):
    vectors = {}
    for x in x_vals:
        distribution = get_distribution(data1,data2,[x, 0], y_span) 
        properties = get_props(distribution)
        if not np.any(np.isnan(properties)):
            vectors[x] = properties
    return vectors

def get_vectors2(data1, data2, x_vals, y_span=12):
    vectors = {}
    for x in x_vals:
        profile = get_profile(data1,data2,[x, 0], y_span) 
        properties = get_profile_props(profile)
        if not np.any(np.isnan(properties)):
            vectors[x] = properties
    return vectors

def get_vectors3(data1, data2, data3, x_vals, y_span=12):
    vectors = {}
    for x in x_vals:
        profile = get_profile(data1,data3,[x, 0], y_span) 
        properties = (get_profile_props(profile))
        distribution = get_distribution(data2,data3,[x, 0], y_span) 
        properties = np.hstack((properties, get_props(distribution)))
        if not np.any(np.isnan(properties)):
            vectors[x] = properties
    return vectors


def get_distribution(data1, data2, point=(0, 0), y_span=12):
    search_radius = y_span

    indices = data2.query_ball_point(point, search_radius)

    result_points = data1[indices]

    filtered_points = result_points[
        (result_points[:, 0] == point[0]) &
        (result_points[:, 1] >= point[1] - y_span) & (result_points[:, 1] <= point[1] + y_span)
    ]

    return filtered_points[:, 2]


def get_profile(data1, data2, point=(0, 0), y_span=12):
    search_radius = y_span

    indices = data2.query_ball_point(point, search_radius)

    result_points = data1[indices]

    filtered_points = result_points[
        (result_points[:, 0] == point[0]) &
        (result_points[:, 1] >= point[1] - y_span) & (result_points[:, 1] <= point[1] + y_span)
    ]

    y_vals = {}

    for y in range(len(filtered_points[:,1])):
        if filtered_points[y,1] not in y_vals:
            y_vals[filtered_points[y,1]] = []
        y_vals[filtered_points[y,1]].append(filtered_points[y,2])
    

    return y_vals


def get_props(distribution):
    properties = []

    mean = np.mean(distribution)
    variance =  np.var(distribution)
    skw = skew(distribution)
    kurt = kurtosis(distribution)

    
    properties.append(variance)
    properties.append(skw)
    properties.append(kurt)

    norm = np.sum(x**2 for x in properties)**0.5
    
    properties = [x / norm for x in properties]

    
    properties.append(mean)

    z = properties[0]
    y = properties[1]
    x = properties[2]

    roe = np.sqrt(x**2 + y**2 + z**2)
    properties.append((np.atan2(y,x) + (2 * np.pi)) % (2 * np.pi))
    properties.append(np.acos(z/roe)/(np.pi/2))
    #properties.append((np.atan2(np.log(z + 1),y)))

    
    return properties


def get_profile_props(profile):
    if profile:
        y = np.array(sorted(profile.keys()))
        z = np.array([profile[y_val] for y_val in y]).flatten()

        dy = np.gradient(y)
        dz = np.gradient(z)
        d2y = np.gradient(dy)
        d2z = np.gradient(dz)

        curvature = np.abs(dy * d2z - dz * d2y) / (dy**2 + dz**2)**(3/2)

        vec = []
        #vec.append(np.mean(curvature))
        #vec.append(np.log(np.max(np.abs(curvature))))
        vec.append(np.var(curvature))


        vec.append((float)(kurtosis(curvature)))

        vec = np.array(vec)
        mag = np.sqrt(np.sum(vec**2))
        
        if mag > 0:
            vec = vec/mag  

        return list(vec)
    else:
        return []




In [ ]:
# 0 = variance, 1 = skew, 2 = kurtosis, 3 = mean, 4 = azumithol, 5 = polar
properties = [1,5] 
#cluster size, sample size, metric, epsilon, alpha, max eps
parameters = [50, 'chebyshev', 0.054]

fig = plt.figure(figsize=(100,3))

downsampled_indices = np.random.choice(point_cloud.shape[0], size=150000, replace=False)

x_downsampled = point_cloud[downsampled_indices, 0]
y_downsampled = point_cloud[downsampled_indices, 1]
z_downsampled = point_cloud[downsampled_indices, 2]

gs = fig.add_gridspec(2, 2, hspace = 0.8, wspace = 0.3, width_ratios = [8,1] )

ax_scatter = fig.add_subplot(gs[0,0])
ax_scatter.scatter(
x_downsampled,
y_downsampled,
c=z_downsampled,
cmap='binary_r',  
s = 0.25
)

ax_scatter.set_title('LIDAR data heatmap')
ax_scatter.set_xlabel('X-axis')
ax_scatter.set_ylabel('Y-axis')

v_dict = get_vectors(point_cloud,point_tree,x_points, y_span = 10)
v_array = np.array(list(v_dict.values()))
x_values = list(v_dict.keys())

dbscan = DBSCAN(min_samples = parameters[0], 
                metric = parameters[1], 
                eps = parameters[2],
                n_jobs = -1)

labels = dbscan.fit_predict(v_array[:, properties])




defined_indices = np.where(np.logical_or(labels == 0, labels == 2))[0]

core_type_dictionary = {x: labels[i] for i,x in enumerate(v_dict.keys())}


semi_defined_indices = np.where(labels == 1)[0]

core_points = [v_dict[x] for i, x in enumerate(v_dict.keys()) if i in defined_indices and i in dbscan.core_sample_indices_]
core_points = np.array(core_points)[:, [1, 5]]
core_points_labels = labels[[i for i in defined_indices if i in dbscan.core_sample_indices_]]

colors = list(f"C{label + 1}" if (label == 0 or label == 2) else (0,0,0,0) for label in labels)

bar_plot = fig.add_subplot(gs[1,0])

bar_plot.bar(x_values, height=1,width = 2, color=colors, edgecolor="none")

bar_plot.set_title("Core Type by X-position")
bar_plot.set_xlabel("X-Value")
bar_plot.legend(title="Core Type", bbox_to_anchor=(1,2), loc="upper left")
bar_plot.set_yticks([])

ax = fig.add_subplot(gs[1,1])



x_vals = [v_dict[x][1] for x in v_dict.keys()]
y_vals = [v_dict[x][5] for x in v_dict.keys()]

ax.scatter(x_vals, y_vals, color=colors, s=1)

plt.show()

In [ ]:
def convolved_data(data,kernels):
    x_values = np.unique(data[:,0])
    y_values = np.unique(data[:,1])
    array = np.zeros((len(y_values), len(x_values)))
    map = {(row[0],row[1]): (index, row[2]) for index,row in enumerate(data)}

    for x in range (len(x_values)):
        for y in range (len(y_values)):
            x_val = x_values[x]
            y_val = y_values[y]
            z = map.get((x_val,y_val))
            if z is not None:
                array[y][x] = z[1]

    result_array = np.zeros((len(y_values), len(x_values)))
    
    for kernel in kernels:
        result_array += np.abs(convolve(array,kernel))

    return_data = data.copy()

    for x in range (len(x_values)):
        for y in range (len(y_values)):
            x_val = x_values[x]
            y_val = y_values[y]
            index = map.get((x_val,y_val))
            if index is not None:
                return_data[index[0],2] = result_array[y][x]
    return return_data

def canny(data):
    sobel_x = np.array([
        [-1, 0, 1],
        [-2, 0, 2],
        [-1, 0, 1]
    ])
    sobel_y = np.array([
        [ 1, 2, 1],
        [ 0, 0, 0],
        [-1,-2,-1]
    ])
    angs = np.array([0, np.pi/4, np.pi/2, 3*np.pi/4])

    x_values = np.unique(data[:,0])
    y_values = np.unique(data[:,1])
    array = np.zeros((len(y_values), len(x_values)))
    map = {(row[0],row[1]): (index, row[2]) for index,row in enumerate(data)}

    for x in range (len(x_values)):
        for y in range (len(y_values)):
            x_val = x_values[x]
            y_val = y_values[y]
            z = map.get((x_val,y_val))
            if z is not None:
                array[y][x] = z[1]

    y_grad = np.zeros((len(y_values), len(x_values)))
    x_grad = np.zeros((len(y_values), len(x_values)))
    NMS_array = np.zeros(array.shape, dtype=object)

    
    y_grad = convolve(array,sobel_y)
    x_grad = convolve(array,sobel_x)


    for x in range (len(x_values)):
        for y in range (len(y_values)):
            Gx = x_grad[y][x]
            Gy = y_grad[y][x]

            Gtheta  = np.atan2(Gy,Gx) % np.pi
            
            theta = np.argmin(np.abs(Gtheta - angs))

            NMS_array[y][x] = tuple((np.sqrt(Gx**2 + Gy**2),theta))

    return_data = data.copy()
    
    for x in range (1,len(x_values) - 1):
        for y in range (1,len(y_values) - 1):
            index = map.get((x_values[x],y_values[y]))
            if index is not None:
                i = index[0]
                mag = (NMS_array[y][x])[0]
                ang = (NMS_array[y][x])[1]
                
                if (ang == 0):
                    n1 = (NMS_array[y][x + 1])[0]
                    n2 = (NMS_array[y][x - 1])[0]
                if (ang == 1):
                    n1 = (NMS_array[y - 1][x - 1])[0]
                    n2 = (NMS_array[y + 1][x + 1])[0]
                if (ang == 2):
                    n1 = (NMS_array[y + 1][x])[0]
                    n2 = (NMS_array[y - 1][x])[0]
                if (ang == 3):
                    n1 = (NMS_array[y - 1][x + 1])[0]
                    n2 = (NMS_array[y + 1][x - 1])[0]
                if mag >= n1 and mag >= n2:
                    return_data[i,2] = mag
                else:
                    return_data[i,2] = 0


    return_data[:, 2] = np.where(return_data[:, 2] <= 50, return_data[:, 2], 50)
    
    return return_data 


kernels = []

smooth_kernels = []

#kernels.append(np.array([[1],[0],[0],[0],
#                         [-4],[0],[0],[0],
#                         [6],[0],[0],[0],
#                         [-4],[0],[0],[0],
#                         [1]]))


kernels.append(np.array([[-1],[-2],[0],[2],[1]]))
smooth_kernels.append(np.array([
    [1,2,1],
    [2,4,2],
    [1,2,1]
]))


#kernels.append(np.array([[1,-4,6,-4,1]]))
#
#smooth_kernels.append(np.array([[1,1,1,1,1]]))





#point_cloud_convolved = convolved_data(point_cloud,smooth_kernels)

point_cloud_convolved = canny(convolved_data(point_cloud,smooth_kernels))



In [ ]:
downsampled_indices = np.random.choice(point_cloud.shape[0], size=1000000, replace=False)
x_downsampled = point_cloud[downsampled_indices, 0]
y_downsampled = point_cloud[downsampled_indices, 1]
z_downsampled = point_cloud[downsampled_indices, 2]

x_downsampled2 = point_cloud_convolved[:, 0]
y_downsampled2 = point_cloud_convolved[:, 1]
z_downsampled2 = point_cloud_convolved[:, 2]


fig, axs = plt.subplots(2, 2, figsize=(150, 6), gridspec_kw={'width_ratios': [1, 0.05]}, dpi = 300)

sc1 = axs[0, 0].scatter(
    x_downsampled, y_downsampled, c=z_downsampled, cmap='viridis', s=0.3
)
axs[0, 0].set_title('LIDAR data heatmap')
axs[0, 0].set_xlabel('X-axis')
axs[0, 0].set_ylabel('Y-axis')

cbar1 = fig.colorbar(sc1, cax=axs[0, 1])
cbar1.set_label('Z Value')

axs[0, 0].set_xticks(list(range(0, 31001, 1000)))  

axs[0, 0].set_xticklabels(list(range(0, 31001, 1000)), rotation=70)

sc2 = axs[1, 0].scatter(
    x_downsampled2, y_downsampled2, c=z_downsampled2, cmap='viridis_r', s=0.3
)
axs[1, 0].set_title('LIDAR data heatmap, convolved')
axs[1, 0].set_xlabel('X-axis')
axs[1, 0].set_ylabel('Y-axis')

cbar2 = fig.colorbar(sc2, cax=axs[1, 1])
cbar2.set_label('Z Value')
axs[1, 0].set_xticks(list(range(0, 31001, 1000)))

axs[1, 0].set_xticklabels(list(range(0, 31001, 1000)), rotation=70)

plt.tight_layout()
plt.show()

plt.show()

In [ ]:
x_values = [x for i, x in enumerate(v_dict.keys()) if i not in defined_indices]
indices = [i for i, x in enumerate(v_dict.keys()) if i not in defined_indices]

profiles = {x: get_profile(point_cloud, point_tree, (x, 0)) for x in x_values}



def view_profile(x_position=1000):
   
    fig = plt.figure(figsize=(19, 3))
    plt.ion()
    gs = fig.add_gridspec(2, 1, hspace=0.8, wspace=0.3)
    ax = fig.add_subplot(gs[0, 0])

    ax.bar(x_values, height=1, width=1, color="black")
    x_value = x_values[np.argmin(np.abs(np.array(x_values) - x_position))]
    ax.axvline(x=x_value, color="r")
    prof = fig.add_subplot(gs[1, 0])
    profile = profiles[x_value]
    y_values = list(profile.keys())
    z_values = list(profile.values())
    prof.scatter(y_values, z_values, s=1)
    print(get_profile_props(profile))
    plt.show()

interact(view_profile, x_position=(0, (np.max(x_values))))


In [ ]:
x_values = [x for i, x in enumerate(v_dict.keys()) if i not in defined_indices]
indices = [i for i, x in enumerate(v_dict.keys()) if i not in defined_indices]

profiles2 = {x: get_profile(point_cloud_convolved, point_tree, (x, 0)) for x in x_values}

def view_profile(x_position=1000):
    fig = plt.figure(figsize=(19, 3))
    gs = fig.add_gridspec(2, 1, hspace=0.8, wspace=0.3)
    ax = fig.add_subplot(gs[0, 0])

    ax.bar(x_values, height=1, width=1, color="black")
    x_value = x_values[np.argmin(np.abs(np.array(x_values) - x_position))]
    ax.axvline(x=x_value, color="r")
    prof = fig.add_subplot(gs[1, 0])
    profile = profiles2[x_value]
    y_values = list(profile.keys())
    z_values = list(profile.values())
    prof.scatter(y_values, z_values, s=1)
    print(get_profile_props(profile))
    plt.show()

interact(view_profile, x_position=(0, (np.max(x_values))))

In [ ]:
x_values = [x for i, x in enumerate(v_dict.keys()) if i not in defined_indices]

#x_values = np.unique(point_cloud[:,0])

#v_dict2 = get_vectors2(point_cloud,point_tree,x_values)

v_dict2 = get_vectors(point_cloud_convolved,point_tree,x_values)



#
dimensions = [1,2,3,4,5]

hdbscan_refined = DBSCAN(min_samples = 25,
                             metric = "chebyshev",
                             eps= 1.3,
                             n_jobs=-1)

labels = hdbscan_refined.fit_predict(np.array(list(v_dict2.values()))[:,dimensions])

core_type_dictionary.update({x: labels[i] + 4 for i,x in enumerate(v_dict2.keys())})

In [ ]:
fig = plt.figure(figsize=(60,3))

gs = fig.add_gridspec(2, 1 )


colors = [f"C{label}"  if label >= 0 else (0,0,0,0)  for label in labels]

bar_plot = fig.add_subplot(gs[1,0])

ax = fig.add_subplot(gs[0,0])

x_vals = [x for x in v_dict2.keys()]
y_vals = [v_dict2[x][0] for x in v_dict2.keys()]

ax.scatter(x_vals, y_vals,color = colors, s=1)


ax.set_xticks(list(range(0, 33001, 1000)))  

ax.set_xticklabels(list(range(0, 33001, 1000)), rotation=70)

legend_labels = np.unique(labels)
legend_colors = [f"C{label}"  if label >= 0 else (0,0,0,0) for label in legend_labels]
legend_handles = [mpatches.Patch(color=color, label=label) for label, color in zip(legend_labels, legend_colors)]

bar_plot.bar(x_values, height=1.5, width=2, color=colors, edgecolor="none")

bar_plot.legend(handles=legend_handles, title="Core Type", bbox_to_anchor=(1,1), loc="upper left")

bar_plot.set_title("Core Type by X-position")
bar_plot.set_xlabel("X-Value")
bar_plot.set_yticks([])
bar_plot.set_xticks(list(range(0, 31001, 1000)))  

bar_plot.set_xticklabels(list(range(0, 31001, 1000)), rotation=70)

plt.show()

In [ ]:
fig = plt.figure(figsize=(150,3), dpi = 500)

core_types = list(core_type_dictionary.values())
x_positions = list(core_type_dictionary.keys())

colors2 = [f"C{type}" if type >= 0  else (0,0,0,0) for type in core_types]

ax = fig.add_subplot(111)


ax.bar(x_positions, height=32, width= 1, color=colors2, edgecolor="none", bottom = -16)
ax.scatter( point_cloud[:, 0], point_cloud[:, 1], c=point_cloud[:, 2], cmap='binary', s=0.08, alpha = 0.4)


legend_labels2 = np.unique(core_types)
legend_colors2 = [f"C{label}" if label >= 0 else (0,0,0,0)  for label in legend_labels2 ]
legend_handles2 = [mpatches.Patch(color=color, label=label) for label, color in zip(legend_labels2, legend_colors2)]

ax.set_xticks(list(range(0, 31001, 500)))
ax.set_xticklabels(list(range(0, 31001, 500)), rotation=70)
ax.legend(handles=legend_handles2, title="Core Type", bbox_to_anchor=(1,1), loc="upper left")

plt.show()

In [ ]:
vectors = np.array(list(v_dict2.values()))
x = vectors[:,5]
y = vectors[:,2]
z = vectors[:,4]


fig = go.Figure(data=[go.Scatter3d(
    x=x, y=y, z=z,
    mode='markers',
    marker=dict(size=1.3, color=labels, opacity=1),
)])

fig.update_layout(
    title="3D Scatter of Vectors by Core Type",
    scene=dict(
        xaxis_title="0",
        yaxis_title="1",
        zaxis_title="2",
        camera=dict(projection=dict(type="orthographic"))
    ),
    width=1000,
    height=1000
)

fig.show()

In [ ]:
paths2 = find_parts(base_path, "Box1\AdaptiveZ_10mm")

In [ ]:
point_clouds = []

x_offset = 0

for path in paths2:
    temp_cloud, x_start, x_stop = get_point_cloud(path)
    x_offset -= x_stop
    temp_cloud[:,0] += x_offset
    x_offset += x_start
    point_clouds.append(temp_cloud)
   
point_cloud2 = np.vstack(point_clouds)
x_values = np.unique(point_cloud2[:,0])

print(f"total # of point {point_cloud.shape[0]}")

point_tree2 = KDTree(point_cloud2[:,:2])

In [ ]:

c_points = core_points
c_type = core_points_labels

type_dict = {i: c_type[i] for i in range(len(c_points))}

tree2 = KDTree(c_points)

v_dict = get_vectors(point_cloud2,point_tree2,x_values)

indices = [tree2.query([v[1], v[5]], k=1, distance_upper_bound=0.1)[1] for v in v_dict.values()]

labels = np.array([type_dict[index] if index != len(c_points) else -1 for index in indices])

prediction = {x: label for x, label in zip(v_dict.keys(), labels)}

x_vals = [v_dict[x][1] for x in v_dict.keys()]
y_vals = [v_dict[x][5] for x in v_dict.keys()]
colors = [f"C{prediction[x] + 2}" if prediction[x] >= 0 else "black" for x in v_dict.keys()]



fig = plt.figure(figsize=(19,3))

downsampled_indices = np.random.choice(point_cloud2.shape[0], size=50000, replace=False)

x_downsampled = point_cloud2[downsampled_indices, 0]
y_downsampled = point_cloud2[downsampled_indices, 1]
z_downsampled = point_cloud2[downsampled_indices, 2]


gs = fig.add_gridspec(2, 2, hspace = 0.5, wspace = 0.2, width_ratios = [9,1] )

ax_scatter = fig.add_subplot(gs[0,0])
ax_scatter.scatter(
    x_downsampled,
    y_downsampled,
    c=z_downsampled,
    cmap='viridis',
    s = 1
)

ax_scatter.set_title('LIDAR data heatmap')
ax_scatter.set_xlabel('X-axis')
ax_scatter.set_ylabel('Y-axis')
  
bar_plot = fig.add_subplot(gs[1,0])

bar_plot.bar(v_dict.keys(), height=1, width=1, color=colors, edgecolor="none")

bar_plot.set_title("Core Type by X-position")
bar_plot.set_xlabel("X-Value")
bar_plot.set_yticks([])

ax = fig.add_subplot(gs[:,1])

ax.scatter(x_vals, y_vals, color=colors, s=1)

plt.show()